# Interactive Use

The documentation already discusses general installation (FIXME: link) of TOAST with conda or pip, so we will not cover that here.  Instead this notebook focuses on considerations for running interactively.

## Local Serial Use

On a local laptop or workstation, it should be straightforward to install the additional tools needed for these notebooks with either conda or pip:

    conda install -c conda-forge jupyter-notebook wurlitzer ipywidgets plotly plotly-resampler
    
OR

    python3 -m pip install jupyter-notebook wurlitzer ipywidgets plotly plotly-resampler

## Local Parallel Use

In addition to the packages needed in the last section, many parts of these notebooks can make use of MPI for parallelism if it is available.  To enable MPI with toast from inside a jupyter notebook, you will first need the mpi4py package.  If you are using `conda` to manage your environment, doing:

    conda install mpi4py
    
**may** install a working package (this will install conda packages for MPI).  If you are using a virtualenv and installing packages with `pip`, then first ensure that your system has a working MPI installation (try typing `which mpicc`).  Then make sure your environment is activated and do:

    python3 -m pip install mpi4py

Test that you can import mpi4py and that every process has a unique rank:

    mpirun -np 4 python3 -c 'from mpi4py import MPI; print(f"hello from rank {MPI.COMM_WORLD.rank}")'
    
If that is working, then install the `ipyparallel` package:

    conda install ipyparallel
    
OR

    python3 -m pip install ipyparallel

Next we need to enable this parallel extension in jupyter:

    jupyter serverextension enable --py ipyparallel
    
**IMPORTANT**:  Before enabling MPI, decide how many MPI processes you want to use.  This should not be more than the number of physical cores on your system.  You should be sure to set the number of OpenMP threads per process such that the number of threads times the number of processes is not more than the physical cores on your system.  For example, if you have 8 physical cores and want to use 4 MPI processes, then you should set the number of OpenMP threads to be no more than 2.  If you do not fail to do this, you may find that every MPI process is using all cores on the system, leading to the notebook (and your system) running very slow.

### Example:  8 cores, 4 MPI processes, 2 threads per process

Set the number of threads **before** launching jupyter:

    export OMP_NUM_THREADS=2
    jupyter-notebook
    
And use `n=4` in the `Cluster()` constructor below.

### Example:  8 cores, 4 MPI processes, 1 thread per process

This might be a conservative choice where you don't want to use the entire resources of the server / laptop.  Set the number of threads **before** launching jupyter:

    export OMP_NUM_THREADS=1
    jupyter-notebook
    
And use `n=4` in the `Cluster()` constructor below.

### Example:  8 cores, 8 MPI processes, 1 thread per process

In general, toast parallelizes better with MPI than relying on threading.  Set the number of threads **before** launching jupyter:

    export OMP_NUM_THREADS=1
    jupyter-notebook
    
And use `n=8` in the `Cluster()` constructor below.

## Computing Centers

Many large computing centers (e.g. NERSC) offer some kind of jupyter service running on dedicated nodes.  These nodes might be shared or their may be other mechanisms for launching ipyparallel processes through the batch system.  This kind of custom setup is beyond the scope of this tutorial.  For NERSC, [see the online documentation](https://docs.nersc.gov/services/jupyter/) for setting up a custom conda environment with your own set of packages (like toast) and then creating kernel spec file from that which can be used with notebooks.

## Imports

If you are using MPI, be sure to initialize the ipyparallel cluster at the start of the notebook, before other imports:

In [ ]:
import ipyparallel as ipp
cluster = ipp.Cluster(engines="mpi", n=4)
client = cluster.start_and_connect_sync()
client.block = True
# We can turn on automatic use of MPI with:
%autopx

Now import all of the remaining packages you want to use.  For example:

In [ ]:
# Built-in modules
import os
import sys

# External modules
import toast
import toast.ops
import toast.widgets

# Capture C++ output in the jupyter cells
import wurlitzer
%load_ext wurlitzer

# Display inline plots
%matplotlib inline

The notebooks in these tutorials also import some helper functions in a file located in the same directory:

In [ ]:
# Load helper tools for the docs
from toast_docs import create_outdir

The following utility function in toast gets information about the MPI environment if it is in use, otherwise it returns `None` for the communicator, `1` for the number of processes, and `0` for the rank of the current process.  These notebooks use the `if rank == 0:` guard around some operations that only need to be done on one process.  If you are not using MPI in your own notebook, you can skip those kinds of checks.

In [ ]:
# Get our MPI world rank to use later when only one process needs to do something,
# like printing info or making a plot.
comm, procs, rank = toast.get_world()
print(f"rank {rank} of {procs} processes in comm world {comm}")

## Runtime Environment

The `toast` module can be influenced by a few environment variables, which must be set **before** importing `toast`.  You can see these in the doc string for the package:

In [ ]:
if rank == 0:
    help(toast)

You can get the current TOAST runtime configuration from the "Environment" class.

In [ ]:
if rank == 0:
    env = toast.Environment.get()
    print(env)

The logging level can be changed by either setting the `TOAST_LOGLEVEL` environment variable to one of the supported levels (`VERBOSE`, `DEBUG`, `INFO`, `WARNING`, `ERROR`, `CRITICAL`) or by using the `set_log_level()` method of the `Environment` class.  The maximum number of threads is controlled by the standard `OMP_NUM_THREADS` environment variable.